In [13]:
import pandas as pd
from  helpers_db import get_engine, run_sql
from backtesting import Backtest, Strategy
from sqlalchemy.sql import text

In [29]:
engine = get_engine()
with engine.begin() as conn:
  data = run_sql(conn, """SELECT dt as datetime, open AS Open, close AS Close, high AS High, low AS Low, volume AS Volume
                          FROM ticks_4h
                          WHERE symbol ='ADAUSD' AND dt >= '2024-01-01'::DATE
                          ORDER BY dt""")
  df = pd.DataFrame(data)
  df.set_index("datetime", inplace = True)
  df.rename(columns={"close": "Close", "open": "Open", "low": "Low", "high": "High", "volume": "Volume"}, inplace=True)
print(df)

                              Close      High       Low      Open  \
datetime                                                            
2024-01-01 00:00:00+00:00  0.596372  0.602324  0.591683  0.593873   
2024-01-01 04:00:00+00:00  0.599317  0.599484  0.590758  0.595044   
2024-01-01 08:00:00+00:00  0.603014  0.604108  0.597756  0.599316   
2024-01-01 12:00:00+00:00  0.605802  0.606815  0.600569  0.603367   
2024-01-01 16:00:00+00:00  0.618768  0.619151  0.602240  0.605850   
...                             ...       ...       ...       ...   
2024-04-13 17:00:00+00:00  0.451668  0.503959  0.439416  0.502804   
2024-04-13 21:00:00+00:00  0.445144  0.454869  0.400001  0.451605   
2024-04-14 01:00:00+00:00  0.446300  0.448764  0.433500  0.445726   
2024-04-14 09:00:00+00:00  0.456652  0.472072  0.446660  0.446660   
2024-04-14 13:00:00+00:00  0.466216  0.467358  0.456580  0.457425   

                                 Volume  
datetime                                 
2024-01-01 00:00:0

In [30]:
from backtesting import Backtest, Strategy
# from backtesting.lib import crossover
# from backtesting.test import SMA

class SmaCross(Strategy):
    n1 = 16
    n2 = 64

    # def init(self):
    #     close = self.data.Close
    #     # self.sma1 = self.I(SMA, close, self.n1)
    #     # self.sma2 = self.I(SMA, close, self.n2)

    def next(self):
        if self.signal_buy:
            self.buy()
        if self.signal_sell:
            self.sell()
        # if crossover(self.sma1, self.sma2):
        #     self.buy()
        # elif crossover(self.sma2, self.sma1):
        #     self.sell()


bt = Backtest(df, SmaCross, cash=10000, commission=.002, exclusive_orders=True)

output = bt.run()
bt.plot()
print(output)

/opt/homebrew/Caskroom/miniforge/base/lib/python3.10/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/opt/homebrew/Caskroom/miniforge/base/lib/python3.10/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/opt/homebrew/Caskroom/miniforge/base/lib/python3.10/site-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
/opt/homebrew/Caskroom/miniforge/base/lib/python3.10/site-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


Start                     2024-01-01 00:00...
End                       2024-04-14 13:00...
Duration                    104 days 13:00:00
Exposure Time [%]                   88.977636
Equity Final [$]                 12979.545923
Equity Peak [$]                   13206.33915
Return [%]                          29.795459
Buy & Hold Return [%]              -21.824633
Return (Ann.) [%]                  147.577739
Volatility (Ann.) [%]              203.810415
Sharpe Ratio                         0.724093
Sortino Ratio                        3.515924
Calmar Ratio                         7.323765
Max. Drawdown [%]                  -20.150529
Avg. Drawdown [%]                   -8.541453
Max. Drawdown Duration       32 days 00:00:00
Avg. Drawdown Duration        7 days 17:00:00
# Trades                                   12
Win Rate [%]                        66.666667
Best Trade [%]                      21.150943
Worst Trade [%]                     -8.956109
Avg. Trade [%]                    